In [ ]:
import requests
import json
import re
from bs4 import BeautifulSoup
import re
import emoji
from datetime import datetime


tag_list=[]
Page_Title={}

#tset url:https://blog.plain-me.com/page/2/

def Set_Data(url,Type,tag_list):
    href=[]
    Title=''
    Author=''
    Date=''
    text=''
    Number=1
    count=0
    while (True):
        r=requests.get(re.sub(r'page/\d+', f'page/{Number}', url)) #用正則表達式處理網址
        Number+=1
        soup=BeautifulSoup(r.text,"html.parser")
        sel=soup.select("body.home.blog div.fusion-row div.fusion-post-wrapper div.fusion-alignleft a")
        if (not sel): break
        for i in sel:
            text='' #初始化合併過的內文
            page=i.get('href')
            page_r=requests.get(page) #開啟前面select/get到的網址
            page_soup=BeautifulSoup(page_r.text,"html.parser")

            #由於存在沒有標題的文章，導致IndexError，直接將空標題文章跳過。
            try:
                Title=page_soup.select("body.post-template-default div.fusion-wrapper div.fusion-page-title-captions h1")[0].text
            except IndexError:
                continue

            Author=page_soup.select("body.post-template-default div.fusion-row span.fn a")[0].text
            Date=page_soup.select("body.post-template-default div.fusion-row div.fusion-meta-info span:not([class])")[0].text
            Date=datetime.strptime(Date, "%d %m 月, %Y")
            Date= Date.strftime("%Y-%m-%d")
            text=page_soup.select('div.post-content')[0].text
            text=re.sub(r'[^\u4e00-\u9fff，。、；：「」『』（）《》？！\s]', '', text) #去除非中文常用之特殊符號
            text=re.sub('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+','',text)
            Page_data={'Title':Title,'Type':Type,'Author':Author,'Date' : Date,'Body' : text,'Url' : page}
            Page_Title[Title]=Page_data

            count+=1
            print("NO."+str(count)+" "+Title)

while(1):
    print("---貼上要爬蟲頁面的網址，要帶有頁碼且頁碼隨意，但只會自動從第一頁開始，輸入為空則結束\n---",end='')
    url=input()
#     url='https://blog.plain-me.com/page/2/'
    if (not url): break
    print("---輸入爬取的網頁資料類型，食|衣|住|行|生活|育樂|醫療|養生\n---",end='')
    Type=input()
    print("---輸入要忽略的文章tag，可依次輸入多個，輸入為空則結束\n---",end='')
    while (True):
        tag_add=input()
        if (tag_add):
            tag_list.append(tag_add)
            print("---",end='')
        else:
            break
    print("---開始爬取---")
    Set_Data(url,Type,tag_list)
    
print("---輸入創建的json檔名，預設為'Crawled_from_Plain-me'\n",end='')
name=input()
if (not name):
    name="Crawled_from_Plain-me"
    print(name)
with open(name+".json","w",encoding='utf-8')as f:
    json.dump(Page_Title,f,indent=4,ensure_ascii=False) #存檔
print("---結束---")
